### Sieć neuronowa

Sieć neuronowa pozwala nam na stworzenie i wyszkolenie modelu, który będzie się starał jak najlepiej przewidzieć kategorię dla danej wejściowej.

W naszym przypadku użyjemy sieci składającej się z 6 warstw:
1. Warstwa spłaszczająca - zamienia nam wielowymiarowe dane na jednowymiarową kolumnę.
2. Warstwa gęsta? 512 neuronów - każdy neuron tej warstwy jest połączony z każdym poprzedniej.
3. Warstwa odrzucająca - zapobiega zbytniemu uczeniu się sieci pod dany zestaw testów.
4. Warstwa gęsta? 128 neuronów - 
5. Warstwa odrzucająca
5. Warstwa gęsta? 10 neuronów - finalna warstwa zawierająca tylko 10 neuronów, czyli tyle ile mamy kategorii. Jest to wyjście z naszego modelu. Im większa wartość neuronu tym pewniejsza jest sieć, że jest to ta kategoria.

#### Źródła
- https://www.tutorialexample.com/understand-dense-layer-fully-connected-layer-in-neural-networks-deep-learning-tutorial/
- https://www.tensorflow.org/datasets/keras_example
- https://www.theprofessionalprogrammer.com/2018/11/neural-network-dense-layers.html
- https://iq.opengenus.org/dense-layer-in-tensorflow/

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
    """Zamieniamy pixele z watrości 0 - 255 na wartości 0.0 - 1.0"""
    return tf.cast(image, tf.float32) / 255., label

# num_parallel_calls - pozwala na szybsze wykonanie map poprzez równoległe mapowanie wielu elementów
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
# shuffle bierze buffer_size który ustawiamy na wielkość zbioru żeby na pewno było dobrze pomieszane
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
# dzielenie na części dla każdej generacji
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

# model.summary()

In [ ]:
from skimage import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

categories = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

our_nums = ["test_0", "test_7"]

for i in range(len(our_nums)):
    image = io.imread(f"{our_nums[i]}.png", as_gray=1)
    image_reshaped = image.reshape(-1, 28, 28, 1)

    prediction = model.predict(image_reshaped)

    plt.figure(figsize=(8,5))

    plt.subplot(1, 2, 1)
    plt.title(f"Prediction = {categories[np.argmax(prediction[0])]}", fontdict={"fontsize": 20})
    io.imshow(image)

    plotable = pd.DataFrame(prediction[0])

    plt.subplot(1, 2, 2)
    plt.tight_layout()
    plt.xticks(categories)
    plt.bar(categories, plotable[0])

    plt.show()

